Dataset
-- https://grouplens.org/datasets/movielens/

In [1]:
!pip install torch

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [3]:
movies = pd.read_csv('../data/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
movies.head(10)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [4]:
users = pd.read_csv('../data/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users.head(10)

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [5]:
ratings = pd.read_csv('../data/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings.head(10)

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


### Split Train Test

In [6]:
training_set = pd.read_csv('../data/ml-100k/u1.base', delimiter = '\t', header = None)
training_set.head(10)

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
5,1,7,4,875071561
6,1,8,1,875072484
7,1,9,5,878543541
8,1,11,2,875072262
9,1,13,5,875071805


In [7]:
# Convert Dataframe to array
training_set = np.array(training_set, dtype = 'int')

In [8]:
test_set = pd.read_csv('../data/ml-100k/u1.test', delimiter = '\t', header = None)
test_set.head(10)

,0,1,2,3
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198
5,1,20,4,887431883
6,1,23,4,875072895
7,1,24,3,875071713
8,1,27,2,876892946
9,1,31,3,875072144


In [9]:
# Convert Dataframe to array
test_set = np.array(test_set, dtype = 'int')

In [10]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))
(nb_users, nb_movies)

(943, 1682)

In [11]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data


In [12]:
training_set = convert(training_set)
test_set = convert(test_set)

### Convert data into Torch tensors


In [13]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

### Converting the ratings to binary ratings: 1 - liked, 0 - not liked

In [14]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

## Build RBM

In [40]:
class RBM():
    
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.bias_a = torch.randn(1, nh)
        self.bias_b = torch.randn(1, nv)
    
    def sample_hidden(self, x):
        Wx = torch.mm(x, self.W.t())
        activation = Wx + self.bias_a.expand_as(Wx)
        prob_h_given_v = torch.sigmoid(activation)
        return prob_h_given_v, torch.bernoulli(prob_h_given_v)
    
    def sample_visible(self, y):
        Wy = torch.mm(y, self.W)
        activation = Wy + self.bias_b.expand_as(Wy)
        prob_v_given_h = torch.sigmoid(activation)
        return prob_v_given_h, torch.bernoulli(prob_v_given_h)
    
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.bias_b += torch.sum((v0 - vk), 0)
        self.bias_a += torch.sum((ph0 - phk), 0)
        
        

In [41]:
num_vis = len(training_set[0])
num_hid = 100
batch_size = 100

In [42]:
rbm = RBM(num_vis, num_hid)

## Training the Restricted Boltzmann Machine

In [45]:
num_epoch = 10

for epoch_num in range(1, num_epoch + 1):
    train_loss = 0
    counter = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user: id_user + batch_size]
        v0 = training_set[id_user: id_user + batch_size]
        ph0, _ = rbm.sample_hidden(v0)
        for k in range(10):
            _, hk = rbm.sample_hidden(vk)
            _, vk = rbm.sample_visible(hk)
            vk[v0 < 0] = v0[v0 < 0]   
        
        phk, _ = rbm.sample_hidden(vk)
        
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        counter += 1
    print('Epoch: ' + str(epoch_num) + ' loss: ' + str(train_loss/counter))

Epoch: 1 loss: tensor(0.2459)
Epoch: 2 loss: tensor(0.2469)
Epoch: 3 loss: tensor(0.2473)
Epoch: 4 loss: tensor(0.2430)
Epoch: 5 loss: tensor(0.2504)
Epoch: 6 loss: tensor(0.2469)
Epoch: 7 loss: tensor(0.2460)
Epoch: 8 loss: tensor(0.2473)
Epoch: 9 loss: tensor(0.2484)
Epoch: 10 loss: tensor(0.2450)


## Testing the RBM

In [44]:
test_loss = 0
counter = 0.
for id_user in range(0, nb_users):
    v = training_set[id_user: id_user + 1]
    vt = test_set[id_user: id_user + 1]
    if len(vt[vt >= 0]) > 0:
        _, h = rbm.sample_hidden(v)
        _, v = rbm.sample_visible(h)
        
        test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
        counter += 1
print('Test loss: ' + str(test_loss/counter))

Test loss: tensor(0.2552)
